In [1]:
# Import dependencies
import pandas as pd
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf

# Read in data
olympic_df = pd.read_csv("./cleandatabmi.csv")
olympic_df.head()

,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,BMI
0,A Dijiang,M,24,180,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,1,25
1,A Lamusi,M,23,170,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,1,21
2,Christine Jacoba Aaftink,F,21,185,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,1,24
3,Christine Jacoba Aaftink,F,21,185,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,"Speed Skating Women's 1,000 metres",1,24
4,Christine Jacoba Aaftink,F,25,185,82.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,Speed Skating Women's 500 metres,1,24


In [2]:
#Drop non-feature columns
olympic_df.drop(['Name',"Sex", "Height", "Weight", "Team", "NOC", "Games","Year", "Season","City","Sport", "Event"], inplace=True, axis=1)
olympic_df.head()

,Age,Medal,BMI
0,24,1,25
1,23,1,21
2,21,1,24
3,21,1,24
4,25,1,24


In [3]:
#check unique value count for encoding
olympic_df.nunique(axis=0)

Age      61
Medal     2
BMI      46
dtype: int64

In [4]:
#create categorical variable list for encoding
olympic_cat = olympic_df.dtypes[olympic_df.dtypes == "object"].index.tolist()
olympic_cat

[]

In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(olympic_df[olympic_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(olympic_cat)


In [6]:
#Merge encoded values and numbered medals and drop original columns
olympic_df = olympic_df.merge(encode_df,left_index=True, right_index=True)
olympic_df = olympic_df.drop(olympic_cat,1)

olympic_df.head()

,Age,Medal,BMI
0,24,1,25
1,23,1,21
2,21,1,24
3,21,1,24
4,25,1,24


In [7]:
# Split preprocessed data into features and target arrays
y = olympic_df["Medal"].values
X = olympic_df.drop(["Medal"],1).values

# Split the preprocessed data into training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [8]:
# Create StandardScaler instance
scaler = StandardScaler()

# Fit StandardScaler
X_scaler = scaler.fit(X_train)

# Scale data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Define model
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 24        
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 75
Trainable params: 75
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Compile model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [11]:
# Train model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
4832/4832 [==============================] - 10s 2ms/step - loss: -2.5234 - accuracy: 0.8485: 1s 
Epoch 2/50
4832/4832 [==============================] - 10s 2ms/step - loss: -6.4962 - accuracy: 0.8547
Epoch 3/50
4832/4832 [==============================] - 11s 2ms/step - loss: -10.3640 - accuracy: 0.8547 2s -  - ETA: 1s - ETA: 0s - loss: -10
Epoch 4/50
4832/4832 [==============================] - 12s 2ms/step - loss: -14.2304 - accuracy: 0.8547
Epoch 5/50
4832/4832 [==============================] - 10s 2ms/step - loss: -18.1038 - accuracy: 0.8547
Epoch 6/50
4832/4832 [==============================] - 9s 2ms/step - loss: -21.9860 - accuracy: 0.8547
Epoch 7/50
4832/4832 [==============================] - 10s 2ms/step - loss: -25.8518 - accuracy: 0.8547
Epoch 8/50
4832/4832 [==============================] - 11s 2ms/step - loss: -29.7109 - accuracy: 0.8547
Epoch 9/50
4832/4832 [==============================] - 10s 2ms/step - loss: -33.5884 - accuracy: 0.8547
Epoch 10/50
483

In [12]:
# Evaluate model using test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1611/1611 - 2s - loss: -1.9995e+02 - accuracy: 0.8505
Loss: -199.94581604003906, Accuracy: 0.8504714369773865
